In [4]:
# 对DeepSeek-Coder-V2-Instruct_sft.jsonl纠错
import os
import json
import pandas as pd
exist_data_set = set()
data_file = 'data/DeepSeek-Coder-V2-Instruct_sft.jsonl'
if os.path.exists(data_file):
    with open(data_file, 'r') as f:
        lines = f.readlines()
        data = json.loads(' '.join(["["]+lines+["]"]))
        data = pd.DataFrame(data)
data.head()

,instruction,input,output,DeepSeek-Coder-V2-Instruct,answer,id
0,你是一个逻辑推理专家，擅长复杂的推理和反思。以下是一个单项选择题，所有的问题都是（close...,,"<thinking>\n首先，我们需要根据词汇表将每个单词翻译成法文。\n\n1. ""the...",A,A,round1_train_data_000_1
1,你是一个逻辑推理专家，擅长复杂的推理和反思。以下是一个单项选择题，所有的问题都是（close...,,"<thinking>\n首先，我们需要将英文句子 ""the egg falls off th...",A,A,round1_train_data_000_2
2,你是一个逻辑推理专家，擅长复杂的推理和反思。以下是一个单项选择题，所有的问题都是（close...,,<thinking>\n首先，我们需要计算B^2 - 4AC的值。\n\n1. 计算B^2:...,C,C,round1_train_data_001_0
3,你是一个逻辑推理专家，擅长复杂的推理和反思。以下是一个单项选择题，所有的问题都是（close...,,<thinking>\n首先，我们需要计算B^2 - 4AC的值。\n\n1. 代入已知的系...,B,B,round1_train_data_001_1
4,你是一个逻辑推理专家，擅长复杂的推理和反思。以下是一个单项选择题，所有的问题都是（close...,,<thinking>\n首先，我们需要计算B^2 - 4AC的值。\n\n根据题目给出的系数...,B,B,round1_train_data_001_2


In [8]:
data['output'] = data['output'].apply(lambda x: x[x.find('<thinking>\n'):])
data['instruction'] = data['instruction'].apply(lambda x: x[:x.find('\n\n        ### 答案:')])
data.head()

,instruction,input,output,DeepSeek-Coder-V2-Instruct,answer,id
0,你是一个逻辑推理专家，擅长复杂的推理和反思。以下是一个单项选择题，所有的问题都是（close...,,"<thinking>\n首先，我们需要根据词汇表将每个单词翻译成法文。\n\n1. ""the...",A,A,round1_train_data_000_1
1,你是一个逻辑推理专家，擅长复杂的推理和反思。以下是一个单项选择题，所有的问题都是（close...,,"<thinking>\n首先，我们需要将英文句子 ""the egg falls off th...",A,A,round1_train_data_000_2
2,你是一个逻辑推理专家，擅长复杂的推理和反思。以下是一个单项选择题，所有的问题都是（close...,,<thinking>\n首先，我们需要计算B^2 - 4AC的值。\n\n1. 计算B^2:...,C,C,round1_train_data_001_0
3,你是一个逻辑推理专家，擅长复杂的推理和反思。以下是一个单项选择题，所有的问题都是（close...,,<thinking>\n首先，我们需要计算B^2 - 4AC的值。\n\n1. 代入已知的系...,B,B,round1_train_data_001_1
4,你是一个逻辑推理专家，擅长复杂的推理和反思。以下是一个单项选择题，所有的问题都是（close...,,<thinking>\n首先，我们需要计算B^2 - 4AC的值。\n\n根据题目给出的系数...,B,B,round1_train_data_001_2


In [10]:
data.to_json('data/DeepSeek-Coder-V2-Instruct_sft_fix.jsonl', orient='records', indent=4, force_ascii=False)

In [2]:
import json
from collections import Counter
def most_common_element(lst):
    # 使用Counter来统计每个元素的出现次数
    counts = Counter(lst)
    # 返回出现次数最多的元素
    return counts.most_common(1)[0][0]

prefix = 'results_models_merge_reflection_lora_'
# prefix = 'results_models_merge_lora_'
results_file = [f'{prefix}{x}.jsonl' for x in ['Meta-Llama-3.1-8B-Instruct','glm-4-9b-chat','Qwen2-7B-Instruct']]
data_model = {}
data_answer = {}
for result in results_file:
    with open(result) as f:
        for line in f:
            data = json.loads(line)
            for id, question in enumerate(data['questions']):
                idx = f'{data["id"]}_{id}'
                # print(question[result[len(prefix):-6]])
                if idx not in data_model:
                    data_model[idx] = question[result[len(prefix):-6]]
                else:
                    data_model[idx].extend(question[result[len(prefix):-6]])
                data_answer[idx] = question['answer']
true_cnt = 0
for idx, model in data_model.items():
    # print(model)
    if most_common_element(model) == data_answer[idx]:
        true_cnt += 1
print(f'{true_cnt}/{len(data_model)}')

1227/1415
